# Data Exploration

In this notebook we'll look at one of the first elements involved in any Data Engineering project - Getting to know what the inputs might look like.

### Step 1 - What inputs do you have?

In [1]:
%%bash
ls ../data

WDICountry.csv
WDIData.csv
WDISeries.csv


### Step 2 - Get your tools setup

As part of this we will be using [PySpark](http://spark.apache.org/docs/2.1.1/api/python/index.html) to inspect the data on hand and also gather some basic details.

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd

#Locating where pyspark is installed
import findspark
findspark.init()
import pyspark

#Settings for PySpark to work
driver_memory = '4g'
num_executors = 2
executor_memory = '1g'
#pyspark_submit_args = ' --driver-memory ' + driver_memory + ' --executor-memory ' + executor_memory + ' --num-executors ' + num_executors + ' pyspark-shell'
pyspark_submit_args = ' --driver-memory ' + driver_memory + ' pyspark-shell'

#Setting the required parameters to start up PySpark
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#Import Modules Needed for PySpark
from pyspark.sql import SparkSession

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  20, truncate = True):
    if(truncate):
        pd.set_option('display.max_colwidth', 50)
    else:
        pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pd.reset_option('display.max_rows')

In [4]:
#Creating a spark session
spark = SparkSession.builder.appName("Data Exploration").getOrCreate()

22/01/28 20:15:16 WARN Utils: Your hostname, wsc-043-72a resolves to a loopback address: 127.0.1.1; using 192.168.34.43 instead (on interface enp2s0)
22/01/28 20:15:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/28 20:15:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 3 - Look inside your data

We need to look at how our data is composed:
1. Format
2. Structure
3. Size
4. Dimensions

In this example our input is a CSV file with a header.  Let's try to see what the data looks like

#### Read The Data

In [5]:
#Read the file into a Spark Data Frame
country = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../data/WDICountry.csv")

#### Inspect the schema of the file you just read

In [18]:
country.printSchema()

root
 |-- Country Code: string (nullable = true)
 |-- Short Name: string (nullable = true)
 |-- Table Name: string (nullable = true)
 |-- Long Name: string (nullable = true)
 |-- 2-alpha code: string (nullable = true)
 |-- Currency Unit: string (nullable = true)
 |-- Special Notes: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Income Group: string (nullable = true)
 |-- WB-2 code: string (nullable = true)
 |-- National accounts base year: string (nullable = true)
 |-- National accounts reference year: integer (nullable = true)
 |-- SNA price valuation: string (nullable = true)
 |-- Lending category: string (nullable = true)
 |-- Other groups: string (nullable = true)
 |-- System of National Accounts: string (nullable = true)
 |-- Alternative conversion factor: string (nullable = true)
 |-- PPP survey year: string (nullable = true)
 |-- Balance of Payments Manual in use: string (nullable = true)
 |-- External debt Reporting status: string (nullable = true)
 |-- Sys

#### Take a look at some sample data

You can run <dataframe>.show() to look at the sample data.  However the output is not well formatted so we will use our helper function to look at the data.

In [19]:
showDF(country, truncate = False)

Country Code            Short Name            Table Name  \
0           ABW                 Aruba                 Aruba   
1           AFG           Afghanistan           Afghanistan   
2           AGO                Angola                Angola   
3           ALB               Albania               Albania   
4           AND               Andorra               Andorra   
5           ARB            Arab World            Arab World   
6           ARE  United Arab Emirates  United Arab Emirates   
7           ARG             Argentina             Argentina   
8           ARM               Armenia               Armenia   
9           ASM        American Samoa        American Samoa   
10          ATG   Antigua and Barbuda   Antigua and Barbuda   
11          AUS             Australia             Australia   
12          AUT               Austria               Austria   
13          AZE            Azerbaijan            Azerbaijan   
14          BDI               Burundi               Burundi   
15          BEL               Belgium               Belgium   
16          BEN                 Benin                 Benin   
17          BFA          Burkina Faso          Burkina Faso   
18          BGD            Bangladesh            Bangladesh   
19          BGR              Bulgaria              Bulgaria   

                          Long Name 2-alpha code           Currency Unit  \
0                             Aruba           AW           Aruban florin   
1      Islamic State of Afghanistan           AF          Afghan afghani   
2       People's Republic of Angola           AO          Angolan kwanza   
3               Republic of Albania           AL            Albanian lek   
4           Principality of Andorra           AD                    Euro   
5                        Arab World           1A                    None   
6              United Arab Emirates           AE           U.A.E. dirham   
7                Argentine Republic           AR          Argentine peso   
8               Republic of Armenia           AM           Armenian dram   
9                    American Samoa           AS             U.S. dollar   
10              Antigua and Barbuda           AG   East Caribbean dollar   
11        Commonwealth of Australia           AU       Australian dollar   
12              Republic of Austria           AT                    Euro   
13           Republic of Azerbaijan           AZ         New Azeri manat   
14              Republic of Burundi           BI           Burundi franc   
15               Kingdom of Belgium           BE                    Euro   
16                Republic of Benin           BJ  West African CFA franc   
17                     Burkina Faso           BF  West African CFA franc   
18  People's Republic of Bangladesh           BD        Bangladeshi taka   
19             Republic of Bulgaria           BG           Bulgarian lev   

                                                                                                                                                                                                                                                                                                                                                           Special Notes  \
0                                                                                                                                                                                                                          SNA data for 2000-2011 are updated from official government statistics; 1994-1999 from UN databases. Base year has changed from 1995 to 2000.   
1                                                 Fiscal year end: March 20; reporting period for national accounts data is calendar year, estimated to insure consistency between national accounts and fiscal data. National accounts data are sourced from the IMF and differ from the Central Statistics Organization numbers due to exclusion of the opium ec

#### Get Some Basic Stats

In [20]:
#Count the number of records in the dataframe
country.count()

263

#### Examining Dimensions
##### How many different regions do the various countries belong to ?

In [21]:
showDF(country.select('Region').distinct(), truncate = False)

Region
0                  South Asia
1                        None
2          Sub-Saharan Africa
3       Europe & Central Asia
4               North America
5         East Asia & Pacific
6  Middle East & North Africa
7   Latin America & Caribbean

##### How many different income groups do we have across countries?

In [22]:
showDF(country.select('Income Group').distinct(), truncate = False)

Income Group
0  Lower middle income
1                 None
2          High income
3  Upper middle income
4           Low income

#### By applying the same steps as we did for the "WDICountry.csv" dataset, we can see what the rest of the datasets look like

###### WDISeries.csv

In [6]:
series = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../data/WDISeries.csv")

In [24]:
series.printSchema()

root
 |-- Series Code: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Indicator Name: string (nullable = true)
 |-- Short definition: string (nullable = true)
 |-- Long definition: string (nullable = true)
 |-- Unit of measure: string (nullable = true)
 |-- Periodicity: string (nullable = true)
 |-- Base Period: string (nullable = true)
 |-- Other notes: string (nullable = true)
 |-- Aggregation method: string (nullable = true)
 |-- Limitations and exceptions: string (nullable = true)
 |-- Notes from original source: string (nullable = true)
 |-- General comments: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Statistical concept and methodology: string (nullable = true)
 |-- Development relevance: string (nullable = true)
 |-- Related source links: string (nullable = true)
 |-- Other web links: string (nullable = true)
 |-- Related indicators: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- _c20: string (nullable = tru

In [7]:
showDF(series)

22/01/28 20:15:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Series Code, Topic, Indicator Name, Short definition, Long definition, Unit of measure, Periodicity, Base Period, Other notes, Aggregation method, Limitations and exceptions, Notes from original source, General comments, Source, Statistical concept and methodology, Development relevance, Related source links, Other web links, Related indicators, License Type, 
 Schema: Series Code, Topic, Indicator Name, Short definition, Long definition, Unit of measure, Periodicity, Base Period, Other notes, Aggregation method, Limitations and exceptions, Notes from original source, General comments, Source, Statistical concept and methodology, Development relevance, Related source links, Other web links, Related indicators, License Type, _c20
Expected: _c20 but found: 
CSV file: file:///home/aleh/work/data_engineering_workshop/WIBD-Workshops-2018/Data%20Engineering/WDI/data/WDISeries.csv


Series Code                                 Topic  \
0      AG.AGR.TRAC.NO  Environment: Agricultural production   
1   AG.CON.FERT.PT.ZS  Environment: Agricultural production   
2      AG.CON.FERT.ZS  Environment: Agricultural production   
3      AG.LND.AGRI.K2                 Environment: Land use   
4      AG.LND.AGRI.ZS                 Environment: Land use   
5      AG.LND.ARBL.HA                 Environment: Land use   
6   AG.LND.ARBL.HA.PC                 Environment: Land use   
7      AG.LND.ARBL.ZS                 Environment: Land use   
8      AG.LND.CREL.HA  Environment: Agricultural production   
9      AG.LND.CROP.ZS                 Environment: Land use   
10  AG.LND.EL5M.RU.K2                 Environment: Land use   
11  AG.LND.EL5M.RU.ZS                 Environment: Land use   
12  AG.LND.EL5M.UR.K2                 Environment: Land use   
13  AG.LND.EL5M.UR.ZS                 Environment: Land use   
14     AG.LND.EL5M.ZS                 Environment: Land use   
15     AG.LND.FRST.K2                 Environment: Land use   
16     AG.LND.FRST.ZS                 Environment: Land use   
17  AG.LND.IRIG.AG.ZS                 Environment: Land use   
18     AG.LND.PRCP.MM                 Environment: Land use   
19     AG.LND.TOTL.K2                 Environment: Land use   

                                       Indicator Name Short definition  \
0                    Agricultural machinery, tractors             None   
1   Fertilizer consumption (% of fertilizer produc...             None   
2   Fertilizer consumption (kilograms per hectare ...             None   
3                          Agricultural land (sq. km)             None   
4                  Agricultural land (% of land area)             None   
5                              Arable land (hectares)             None   
6                   Arable land (hectares per person)             None   
7                        Arable land (% of land area)             None   
8             Land under cereal production (hectares)             None   
9                 Permanent cropland (% of land area)             None   
10  Rural land area where elevation is below 5 met...             None   
11  Rural land area where elevation is below 5 met...             None   
12  Urban land area where elevation is below 5 met...             None   
13  Urban land area where elevation is below 5 met...             None   
14  Land area where elevation is below 5 meters (%...             None   
15                               Forest area (sq. km)             None   
16                       Forest area (% of land area)             None   
17  Agricultural irrigated land (% of total agricu...             None   
18       Average precipitation in depth (mm per year)             None   
19                                 Land area (sq. km)             None   

                                      Long definition Unit of measure  \
0   Agricultural machinery refers to the number of...            None   
1   Fertilizer consumption measures the quantity o...            None   
2   Fertilizer consumption measures the quantity o...            None   
3   Agricultural land refers to the share of land ...            None   
4   Agricultural land refers to the share of land ...            None   
5   Arable land (in hectares) includes land define...            None   
6   Arable land (hectares per person) includes lan...            None   
7   Arable land includes land defined by the FAO a...            None   
8   Land under cereal production refers to harvest...            None   
9   Permanent cropland is land cultivated with cro...            None   
10  Rural land area below 5m is the total rural la...            None   
11  Rural land area below 5m is the percentage of ...            None   
12  Urban land area below 5m is the total urban la...            None   
13  Urban land area below 5m is the percentage of ...            None   
14  Land area below 5m is the percen

In [26]:
series.count()

1593

#### Examining Dimensions
##### What are the different periodicities or aggregation methods we might expect to see in the data ?

In [27]:
showDF(series.select('Periodicity').distinct(), truncate = False)

Periodicity
0                                                                                                       Annual
1                                                                                                         None
2                                                                            Quarterly (represented as Annual)
3  International Civil Aviation Organization, Civil Aviation Statistics of the World and ICAO staff estimates.

In [28]:
showDF(series.select('Aggregation Method').distinct(), truncate = False)

Aggregation Method
0                                 None
1                     Weighted average
2                       Simple average
3                     Gap-filled total
4                               Median
5                   Unweighted average
6  Linear mixed-effect model estimates
7                     Weighted Average
8                                  Sum

## Exercise

Repeat the same steps for the `WDIData.csv` file and read it into a dataframe called `indicators`.

In [8]:
indicators = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../data/WDIData.csv")

In [32]:
indicators.printSchema()


root
 |-- Country Name: string (nullable = true)
 |-- Country Code: string (nullable = true)
 |-- Indicator Name: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- 1960: double (nullable = true)
 |-- 1961: double (nullable = true)
 |-- 1962: double (nullable = true)
 |-- 1963: double (nullable = true)
 |-- 1964: double (nullable = true)
 |-- 1965: double (nullable = true)
 |-- 1966: double (nullable = true)
 |-- 1967: double (nullable = true)
 |-- 1968: double (nullable = true)
 |-- 1969: double (nullable = true)
 |-- 1970: double (nullable = true)
 |-- 1971: double (nullable = true)
 |-- 1972: double (nullable = true)
 |-- 1973: double (nullable = true)
 |-- 1974: double (nullable = true)
 |-- 1975: double (nullable = true)
 |-- 1976: double (nullable = true)
 |-- 1977: double (nullable = true)
 |-- 1978: double (nullable = true)
 |-- 1979: double (nullable = true)
 |-- 1980: double (nullable = true)
 |-- 1981: double (nullable = true)
 |-- 1982: double (null

In [34]:
showDF(indicators)


Country Name Country Code  \
0    Arab World          ARB   
1    Arab World          ARB   
2    Arab World          ARB   
3    Arab World          ARB   
4    Arab World          ARB   
5    Arab World          ARB   
6    Arab World          ARB   
7    Arab World          ARB   
8    Arab World          ARB   
9    Arab World          ARB   
10   Arab World          ARB   
11   Arab World          ARB   
12   Arab World          ARB   
13   Arab World          ARB   
14   Arab World          ARB   
15   Arab World          ARB   
16   Arab World          ARB   
17   Arab World          ARB   
18   Arab World          ARB   
19   Arab World          ARB   

                                       Indicator Name  \
0   2005 PPP conversion factor; GDP (LCU per inter...   
1   2005 PPP conversion factor; private consumptio...   
2   Access to clean fuels and technologies for coo...   
3             Access to electricity (% of population)   
4   Access to electricity; rural (% of rural popul...   
5   Access to electricity; urban (% of urban popul...   
6   Account ownership at a financial institution o...   
7   Account ownership at a financial institution o...   
8   Account ownership at a financial institution o...   
9   Account ownership at a financial institution o...   
10  Account ownership at a financial institution o...   
11  Account ownership at a financial institution o...   
12  Account ownership at a financial institution o...   
13  Account ownership at a financial institution o...   
14  Account ownership at a financial institution o...   
15  Adequacy of social insurance programs (% of to...   
16  Adequacy of social protection and labor progra...   
17  Adequacy of social safety net programs (% of t...   
18  Adequacy of unemployment benefits and ALMP (% ...   
19  Adjusted net enrollment rate; primary (% of pr...   

              Indicator Code  1960  1961  1962  1963  1964  1965  ...  \
0              PA.NUS.PPP.05   NaN   NaN   NaN   NaN   NaN   NaN  ...   
1          PA.NUS.PRVT.PP.05   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2             EG.CFT.ACCS.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3             EG.ELC.ACCS.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4          EG.ELC.ACCS.RU.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5          EG.ELC.ACCS.UR.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6             FX.OWN.TOTL.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
7          FX.OWN.TOTL.FE.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
8          FX.OWN.TOTL.MA.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
9          FX.OWN.TOTL.OL.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
10         FX.OWN.TOTL.40.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
11         FX.OWN.TOTL.PL.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
12         FX.OWN.TOTL.60.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
13         FX.OWN.TOTL.SO.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
14         FX.OWN.TOTL.YG.ZS   NaN   NaN   NaN   NaN   NaN   NaN  ...   
15  per_si_allsi.adq_pop_tot   NaN   NaN   NaN   NaN   NaN   NaN  ...   
16     per_allsp.adq_pop_tot   NaN   NaN   NaN   NaN   NaN   NaN  ...   
17  per_sa_allsa.adq_pop_tot   NaN   NaN   NaN   NaN   NaN   NaN  ...   
18  per_lm_alllm.adq_pop_tot   NaN   NaN   NaN   NaN   NaN   NaN  ...   
19               SE.PRM.TENR   NaN   NaN   NaN   NaN   NaN   NaN  ...   

         2008       2009       2010       2011       2012       2013  \
0         NaN        NaN        NaN        NaN        NaN        NaN   
1         NaN        NaN        NaN        NaN        NaN        NaN   
2   81.243897  81.844982  82.407647  82.827636  83.169227  83.587141   
3   85.432827  85.189815  86.136134  86.782683  87.288244  88.389705   
4   73.460400  73.541696  74.507470  75.652712  76.628320  78.663736   
5   95.609699  95.088290  95.841533  96.033101  96.060627  96.446582   
6         NaN        NaN        NaN  22.260538        NaN        NaN   
7         NaN        N

In [37]:
indicators.select('Country Code').distinct().count()


264